In [1]:
from datetime import datetime, timedelta
import pytz
import pandas as pd
from tqdm import tqdm
from modeler.modeler import Modeler as m
from processor.processor import Processor as p
from database.market import Market
from database.adatabase import ADatabase
import numpy as np
import matplotlib.pyplot as plt
import pickle
import matplotlib.dates as mdates
import statistics

In [2]:
market = Market()
speculation_db = ADatabase("speculation")
training_year = 4
signal = 0.1
req = 0.1
positions = 5
today = datetime(2022,8,18)
year = today.year
week =  today.isocalendar()[1]
# today = datetime.now()

In [3]:
week

33

In [4]:
market.connect()
sp5 = market.retrieve("sp500")
market.disconnect()

In [5]:
speculation_db.connect()
recs = speculation_db.retrieve('recs')
speculation_db.disconnect()

In [6]:
relevant_recs = recs[(recs["week"]==week) \
                     & (recs["year"]==year) \
                     & (recs["training_year"]==training_year)]

In [7]:
relevant_recs["prediction"] = (relevant_recs["xgb_prediction"] + relevant_recs["skl_prediction"] + relevant_recs["cat_prediction"]) / 3

In [8]:
relevant_recs

,year,week,ticker,training_year,skl_prediction,skl_score,xgb_prediction,xgb_score,cat_prediction,cat_score,prediction
1494,2022,33,MMM,4,150.610623,0.998268,147.624878,0.99676,164.092261,0.917739,154.109254
1495,2022,33,AOS,4,62.288876,0.998268,64.534576,0.99676,63.723778,0.917739,63.515743
1496,2022,33,ABT,4,110.404819,0.998268,109.400787,0.99676,119.878028,0.917739,113.227878
1497,2022,33,ABBV,4,143.179402,0.998268,146.302368,0.99676,120.384186,0.917739,136.621986
1498,2022,33,ABMD,4,291.372336,0.998268,278.221100,0.99676,259.648373,0.917739,276.413936
...,...,...,...,...,...,...,...,...,...,...,...
1987,2022,33,WYNN,4,66.831180,0.998268,68.142555,0.99676,64.626058,0.917739,66.533264
1988,2022,33,XEL,4,75.619537,0.998268,77.647385,0.99676,68.978636,0.917739,74.081853
1989,2022,33,XYL,4,101.256655,0.998268,103.355980,0.99676,89.955445,0.917739,98.189360
1990,2022,33,YUM,4,118.655374,0.998268,117.038528,0.99676,120.517986,0.917739,118.737296


In [9]:
market.connect()
ticker_prices = []
for ticker in tqdm(relevant_recs["ticker"].unique()):
    prices = market.retrieve_ticker_prices("prices",ticker)
    prices = p.column_date_processing(prices)
    prices["year"] = [x.year for x in prices["date"]]
    prices["week"] = [x.week for x in prices["date"]]
    relevant_data = prices[prices["date"]==today]
    ticker_prices.append(relevant_data)
market.disconnect()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 498/498 [00:36<00:00, 13.74it/s]


In [10]:
tp = pd.concat(ticker_prices)

In [11]:
final = tp.groupby(["date","ticker"]).mean().reset_index()
final = final.merge(relevant_recs,on=["year","week","ticker"])
final["delta"] = (final["prediction"] - final["adjclose"]) / final["adjclose"]

In [12]:
todays_rec = final[["date","ticker","adjclose","prediction","delta"]][final["delta"]>=signal].sort_values("delta",ascending=False).head(positions)

In [13]:
todays_rec

,date,ticker,adjclose,prediction,delta
173,2022-08-18,F,16.15,21.301943,0.319006
63,2022-08-18,BKNG,2144.73,2770.457008,0.291751
448,2022-08-18,UAA,9.76,12.068085,0.236484
337,2022-08-18,NWSA,18.28,22.196766,0.214265
307,2022-08-18,MRNA,150.00,175.437291,0.169582


In [14]:
speculation_db.connect()
portfolios = speculation_db.retrieve("portfolios")
speculation_db.disconnect()

In [15]:
if portfolios.index.size < 1:
    positions_dictionary = {}
    portfolio = {"date":today}
    for position in range(positions):
        positions_dictionary[position] = {"cash":100000/positions,"asset":{},"queue":{}}
    portfolio["positions"] = positions_dictionary
else:
    portfolios["positions"] = [pickle.loads(x) for x in portfolios["positions"]]
    portfolio = portfolios.sort_values("date",ascending=False).head(positions).to_dict("records")[0]

In [16]:
existing_assets = [portfolio["positions"][position]["asset"]["ticker"] for position in portfolio["positions"].keys() if "ticker" in portfolio["positions"][position]["asset"].keys()]

In [17]:
portfolio

{'date': Timestamp('2022-08-17 00:00:00'),
 'positions': {0: {'cash': 8.380000000001019,
   'asset': {'date': datetime.datetime(2022, 8, 10, 0, 0),
    'ticker': 'WBD',
    'amount': 1557,
    'price': 13.1},
   'queue': {}},
  1: {'cash': 6.540000000000873,
   'asset': {'date': datetime.datetime(2022, 8, 15, 0, 0),
    'ticker': 'F',
    'amount': 1247,
    'price': 16.18},
   'queue': {}},
  2: {'cash': 23.0199999999968,
   'asset': {'date': datetime.datetime(2022, 8, 15, 0, 0),
    'ticker': 'NLOK',
    'amount': 918,
    'price': 24.19},
   'queue': {}},
  3: {'cash': 8.180000000000291,
   'asset': {'date': datetime.datetime(2022, 8, 15, 0, 0),
    'ticker': 'NCLH',
    'amount': 1447,
    'price': 14.16},
   'queue': {}},
  4: {'cash': 12.319999999999709,
   'asset': {'date': datetime.datetime(2022, 8, 9, 0, 0),
    'ticker': 'NLOK',
    'amount': 814,
    'price': 24.17},
   'queue': {}}}}

In [18]:
## looking for exits
speculation_db.connect()
for position in range(positions):
    position_dictionary = portfolio["positions"][position]
    if len(list(position_dictionary["asset"].keys())) > 0:
        asset = position_dictionary["asset"]
        ticker = asset["ticker"]
        amount = asset["amount"]
        price = asset["price"]
        exit_price = price * (1+req)
        spread = final[final["ticker"]==ticker]
        current_price = spread["high"].item()
        current_gain = (current_price - price) / price
        if spread["high"].item() >= exit_price:
            position_dictionary["cash"] = position_dictionary["cash"] + (exit_price * amount)
            trade = asset
            trade["sell_date"] = today
            trade["sell_price"] = exit_price
            trade["position"] = position
            trade["req"] = req
            trade["training_year"] = training_year
            trade["signal"] = signal
            speculation_db.store("ledger",pd.DataFrame([trade]))
            print("sell",exit_price,spread["high"].item())
            position_dictionary["asset"] = {}
        else:
            if today.weekday() >= 4 and int((today-asset["date"]).days) >= 7:
                print("due date")
                position_dictionary["cash"] = position_dictionary["cash"] + (spread["adjclose"].item() * amount)
                trade = asset
                trade["sell_date"] = today
                trade["sell_price"] = spread["adjclose"].item()
                trade["position"] = position
                trade["req"] = req
                trade["training_year"] = training_year
                trade["signal"] = signal
                speculation_db.store("ledger",pd.DataFrame([trade]))
                position_dictionary["asset"] = {}
            else:
                offerings =  todays_rec[(~todays_rec["ticker"].isin(existing_assets))]
                offerings.sort_values("delta",ascending=False,inplace=True)
                if offerings.index.size > position:
                    offering = offerings.iloc[position]
                    if (current_gain > 0)  and today.weekday() < 2 and (offering["delta"] > (req - current_gain)) and (offering["delta"] >= signal):
                        print("swap")
                        trade = asset
                        trade["sell_date"] = today
                        trade["sell_price"] = current_price
                        trade["position"] = position
                        trade["req"] = req
                        trade["training_year"] = training_year
                        trade["signal"] = signal
                        trade_df = pd.DataFrame([trade])
                        speculation_db.store("ledger",trade_df)
                        position_dictionary["cash"] = position_dictionary["cash"] + current_price * asset["amount"]
                        position_dictionary["asset"] = {}
                        cash = position_dictionary["cash"]
                        position_rec = offering
                        price = position_rec["adjclose"]
                        ticker = position_rec["ticker"]
                        amount = int(cash/price)
                        position_dictionary["queue"] = {"ticker":ticker,"price":price,"amount":amount,"swap":True,"projected_delta":offering["delta"]}
                else:
                    print("hold")
    else:
        print(position_dictionary["cash"])
speculation_db.disconnect()

hold


In [19]:
portfolio

{'date': Timestamp('2022-08-17 00:00:00'),
 'positions': {0: {'cash': 8.380000000001019,
   'asset': {'date': datetime.datetime(2022, 8, 10, 0, 0),
    'ticker': 'WBD',
    'amount': 1557,
    'price': 13.1},
   'queue': {}},
  1: {'cash': 6.540000000000873,
   'asset': {'date': datetime.datetime(2022, 8, 15, 0, 0),
    'ticker': 'F',
    'amount': 1247,
    'price': 16.18},
   'queue': {}},
  2: {'cash': 23.0199999999968,
   'asset': {'date': datetime.datetime(2022, 8, 15, 0, 0),
    'ticker': 'NLOK',
    'amount': 918,
    'price': 24.19},
   'queue': {}},
  3: {'cash': 8.180000000000291,
   'asset': {'date': datetime.datetime(2022, 8, 15, 0, 0),
    'ticker': 'NCLH',
    'amount': 1447,
    'price': 14.16},
   'queue': {}},
  4: {'cash': 12.319999999999709,
   'asset': {'date': datetime.datetime(2022, 8, 9, 0, 0),
    'ticker': 'NLOK',
    'amount': 814,
    'price': 24.17},
   'queue': {}}}}

In [20]:
## looking for entries
for position in range(positions):
    position_dictionary = portfolio["positions"][position]
    if len(list(position_dictionary["queue"].keys())) > 0:
        queue = position_dictionary["queue"]
        ticker = queue["ticker"]
        amount = queue["amount"]
        price = queue["price"]
        spread = final[final["ticker"]==ticker]
        if (price <= spread["high"].item()) and today.weekday() < 2:
            print("buy")
            position_dictionary["asset"] = {"date":today,"ticker":ticker,"amount":amount,"price":price,"swap":queue["swap"],"projected_delta":queue["projected_delta"]}
            position_dictionary["cash"] = position_dictionary["cash"] - amount * price
            position_dictionary["queue"] = {}
        else:
            position_dictionary["queue"] = {}
            print("fail")
    else:
        print(position_dictionary["cash"])

8.380000000001019
6.540000000000873
23.0199999999968
8.180000000000291
12.319999999999709


In [21]:
portfolio

{'date': Timestamp('2022-08-17 00:00:00'),
 'positions': {0: {'cash': 8.380000000001019,
   'asset': {'date': datetime.datetime(2022, 8, 10, 0, 0),
    'ticker': 'WBD',
    'amount': 1557,
    'price': 13.1},
   'queue': {}},
  1: {'cash': 6.540000000000873,
   'asset': {'date': datetime.datetime(2022, 8, 15, 0, 0),
    'ticker': 'F',
    'amount': 1247,
    'price': 16.18},
   'queue': {}},
  2: {'cash': 23.0199999999968,
   'asset': {'date': datetime.datetime(2022, 8, 15, 0, 0),
    'ticker': 'NLOK',
    'amount': 918,
    'price': 24.19},
   'queue': {}},
  3: {'cash': 8.180000000000291,
   'asset': {'date': datetime.datetime(2022, 8, 15, 0, 0),
    'ticker': 'NCLH',
    'amount': 1447,
    'price': 14.16},
   'queue': {}},
  4: {'cash': 12.319999999999709,
   'asset': {'date': datetime.datetime(2022, 8, 9, 0, 0),
    'ticker': 'NLOK',
    'amount': 814,
    'price': 24.17},
   'queue': {}}}}

In [22]:
## looking for recs
for position in range(positions):
    position_dictionary = portfolio["positions"][position]
    ## recommendation impossible
    if (len(list(position_dictionary["queue"].keys())) > 0) or (len(list(position_dictionary["asset"].keys())) > 0) or (position_dictionary["cash"]<=1.0) or (position >= todays_rec.index.size):
        print(position_dictionary["queue"])
    else:
        cash = position_dictionary["cash"]
        position_rec = todays_rec.iloc[position]
        price = position_rec["adjclose"]
        ticker = position_rec["ticker"]
        amount = int(cash/price)
        if amount < 1 or ticker in existing_assets:
            continue
        else:
            position_dictionary["queue"] = {"ticker":ticker,"price":price,"amount":amount,"swap":False,"projected_delta":position_rec["delta"]}
## storing

{}
{}
{}
{}
{}


In [23]:
portfolio["date"] = today

In [24]:
portfolio["positions"] = pickle.dumps(portfolio["positions"])

In [25]:
portfolio

{'date': datetime.datetime(2022, 8, 18, 0, 0),
 'positions': b'\x80\x04\x95w\x02\x00\x00\x00\x00\x00\x00}\x94(K\x00}\x94(\x8c\x04cash\x94\x8c\x15numpy.core.multiarray\x94\x8c\x06scalar\x94\x93\x94\x8c\x05numpy\x94\x8c\x05dtype\x94\x93\x94\x8c\x02f8\x94\x89\x88\x87\x94R\x94(K\x03\x8c\x01<\x94NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00t\x94bC\x08\x00\xf8(\\\x8f\xc2 @\x94\x86\x94R\x94\x8c\x05asset\x94}\x94(\x8c\x04date\x94\x8c\x08datetime\x94\x8c\x08datetime\x94\x93\x94C\n\x07\xe6\x08\n\x00\x00\x00\x00\x00\x00\x94\x85\x94R\x94\x8c\x06ticker\x94\x8c\x03WBD\x94\x8c\x06amount\x94M\x15\x06\x8c\x05price\x94h\x05h\x0bC\x08333333*@\x94\x86\x94R\x94u\x8c\x05queue\x94}\x94uK\x01}\x94(h\x02h\x05h\x0bC\x08\x00`\x8f\xc2\xf5(\x1a@\x94\x86\x94R\x94h\x11}\x94(\x8c\x04date\x94h\x16C\n\x07\xe6\x08\x0f\x00\x00\x00\x00\x00\x00\x94\x85\x94R\x94\x8c\x06ticker\x94\x8c\x01F\x94\x8c\x06amount\x94M\xdf\x04\x8c\x05price\x94h\x05h\x0bC\x08\xaeG\xe1z\x14.0@\x94\x86\x94R\x94uh!}\x94uK\x02}\x94(h\x02h\x05h\x0bC\x08\x00

In [26]:
speculation_db.connect()
speculation_db.store("portfolios",pd.DataFrame([portfolio]))
speculation_db.disconnect()